In [1]:
import requests
response = requests.get('https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json')

if response.status_code == 200:
    print("Successful connection with API.")
    print('-------------------------------')
    data = response.json()
elif response.status_code == 404:
    print("Unable to reach URL.")
else:
    print("Unable to connect API or retrieve data.")
for record in data:
     print("Gender: {},\n Income: {},\n Married: {},\n".format(record['Gender'] , record['Married'],record['Income']))

Successful connection with API.
-------------------------------
Gender: Male,
 Income: No,
 Married: medium,

Gender: Male,
 Income: Yes,
 Married: medium,

Gender: Male,
 Income: Yes,
 Married: low,

Gender: Male,
 Income: Yes,
 Married: low,

Gender: Male,
 Income: No,
 Married: medium,

Gender: Male,
 Income: Yes,
 Married: medium,

Gender: Male,
 Income: Yes,
 Married: low,

Gender: Male,
 Income: Yes,
 Married: low,

Gender: Male,
 Income: Yes,
 Married: medium,

Gender: Male,
 Income: Yes,
 Married: high,

Gender: Male,
 Income: Yes,
 Married: low,

Gender: Male,
 Income: Yes,
 Married: low,

Gender: Male,
 Income: No,
 Married: low,

Gender: Male,
 Income: Yes,
 Married: low,

Gender: Male,
 Income: No,
 Married: medium,

Gender: Female,
 Income: No,
 Married: low,

Gender: Male,
 Income: Yes,
 Married: medium,

Gender: Male,
 Income: Yes,
 Married: medium,

Gender: Male,
 Income: Yes,
 Married: medium,

Gender: Male,
 Income: Yes,
 Married: low,

Gender: Male,
 Income: Yes,
 Ma

In [2]:
# Import necessary libs
import pandas as pd
from pyspark.sql import SparkSession 
from pyspark.sql import functions as sf
from functools import reduce
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime
import time

from pyspark.conf import SparkConf
from pyspark.context import SparkContext

conf = SparkConf().setAppName("DataExtractionApp").setMaster("local[2]")
sc = SparkSession.builder.config("spark.driver.host", "localhost") \
.config ("spark.sql.execution.arrow.enabled", "true").getOrCreate()


Creating a spark dataframe from response json

In [3]:

df_loan = sc.createDataFrame(response.json())
df_loan.show()

+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|Application_ID|Application_Status|Credit_History|Dependents|   Education|Gender|Income|Married|Property_Area|Self_Employed|
+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|      LP001002|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|
|      LP001003|                 N|             1|         1|    Graduate|  Male|medium|    Yes|        Rural|           No|
|      LP001005|                 Y|             1|         0|    Graduate|  Male|   low|    Yes|        Urban|          Yes|
|      LP001006|                 Y|             1|         0|Not Graduate|  Male|   low|    Yes|        Urban|           No|
|      LP001008|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|


Loading extracted loan dataset into SQL

In [4]:
df_loan.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.CDW_SAPP_loan_application") \
.option("user", "root") \
.option("password", "lakshmi") \
.save()